In [ ]:
import os
from threading import Lock


class WriteAheadLog:
    """
    Write-Ahead Log ensures durability for the key-value store by logging all changes to disk.
    """
    def __init__(self, path):
        self.path = os.path.join(path, "wal.log")
        self.lock = Lock()
        os.makedirs(path, exist_ok=True)
        self.file = open(self.path, "a+")

    def write(self, operation, key, value=None):
        """Log operations as OP\tKEY\tVALUE."""
        with self.lock:
            entry = f"{operation}\t{key}\t{value}\n"
            self.file.write(entry if value is not None else f"{operation}\t{key}\n")
            self.file.flush()

    def read_all(self):
        with open(self.path, "r") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) == 3:
                    yield (parts[0], parts[1], parts[2])
                elif len(parts) == 2:
                    yield (parts[0], parts[1], None)